<a href="https://colab.research.google.com/github/muyale/Mental-Health-Assistant-Bot/blob/main/Mental_Health_Assistant_(Bot).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing the necessary dependencies
!pip install transformers
!pip install huggingfacehub
!pip install langchain
!pip install sentence-transformers
!pip install gradio
!pip install chromadb


In [2]:
#importing the downloaded dependencies
import torch
import langchain
import transformers
from langchain import HuggingFaceHub,PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.document_loaders import TextLoader


In [3]:
# creating a hugging face environment variable
import os
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()


··········


In [4]:
# creating the environment variable
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

In [5]:
# CREATING A TEMPLATE
prompt_template = """
{context}

Question: {question}
Answer:"""


prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)


In [6]:
# Getting our LLM
repo_id = 'mistralai/Mistral-7B-Instruct-v0.2'
llm  = HuggingFaceHub(repo_id=repo_id,model_kwargs={"temperature":0.9})
# trying our llm with a bunch of text
llm("What is a miracle?")

# Creating a LLMChain
llm_chain = LLMChain(llm=llm,prompt=prompt)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [7]:
# CREATING A DATABASE FOR OUR LLM TO INDEX
# loading the data
loader = TextLoader('/content/Mental Health Assistant Data.txt')
documents = loader.load()
#splitting the data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=10)
all_splits = text_splitter.split_documents(documents)

In [ ]:
# Getting Embeddings
model_name ='sentence-transformers/all-mpnet-base-v2'
embeddings =HuggingFaceEmbeddings(model_name=model_name)

In [9]:
# Creating a Vector Store
store = Chroma.from_documents(all_splits,embedding=embeddings)

In [10]:
from langchain.chains import RetrievalQA

In [11]:

retriever = store.as_retriever()


qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever,
    chain_type_kwargs = {"prompt": prompt},
    return_source_documents = True,
    verbose = False
)

In [12]:
def mental_health(query):
    result = qa_chain.invoke(query)
    return result


In [13]:
mental_health("WHAT IS STRESS")

{'query': 'WHAT IS STRESS',
 'result': "\nyour emotional response to the worry.\n\ndiscomfort. So if you get tricked into treating worry as a danger, this naturally\nmakes it worse.\n\nworry, how you try to get it under your control. The path out of chronic worry\n\nWhat do you think anxiety is good for? Why do we have the capacity to\nbecome anxious?\n\nQuestion: WHAT IS STRESS\nAnswer: Stress is the emotional and physical feeling of being under strain. It occurs in response to internal or external demands and can lead to a variety of symptoms, including increased heart rate, elevated blood pressure, muscle tension, and feelings of anxiety or unease. Stress can be acute, lasting only a short time, or chronic, lasting for an extended period. The body's stress response is a natural, evolutionary mechanism that helps us respond to threats, but chronic stress can be harmful",
 'source_documents': [Document(page_content='your emotional response to the worry.', metadata={'source': '/content

In [14]:
import textwrap

In [15]:
def wrap_text_preserve_newlines(text, width=700):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])
    return ans

In [17]:
def llm_ans(query):
    llm_response = qa_chain.invoke(query)
    result = process_llm_response(llm_response)
    return result.strip()

In [18]:
llm_ans('what to do when depressed')

"go see a funny movie, or have a nice chat with a friend-I start thinking and\n\ndealing with a different kind of problem-depressive memories of a past event,\n\nI won't be able to solve this because I'll always feel depressed.\n\nmake yourself busy. That's too much like trying to get rid of the thoughts. Not\n\nQuestion: what to do when depressed\nAnswer: Engage in activities that require focus and energy, such as exercising, working on a project, or socializing with friends. These activities can help distract you from your negative thoughts and improve your mood. Additionally, try practicing relaxation techniques, such as deep breathing or meditation, to help calm your mind and reduce stress. If your depression is severe or persistent, consider seeking help from a mental health professional."

In [ ]:
# @title
import gradio as gr
description ='This is a Mental Health Virtual Assistant trained to help you navigate several of your mental health issues.Ask anything about Stress,Depression,Grief,Anxiety.This is not to be subsituted for a therapist ,you are advised to seek professional help where necessary'
gr.Interface(inputs='text',outputs='text',title='MENTAL HEALTH ASSISTANT',description=description,theme=gr.themes.Monochrome(),fn=llm_ans).launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://963ea2023359d237a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
